# Vehicle motion detection using background subtraction.

## Overview

In this article we will be working on the object's motion detection (specifically vehicles) by using the techniques of **contours** detection and also **background subtraction**. In this particular we will be using a video of vehicles for the car detection i.e. it is gonna be a real time experience.

## Application of motion detection

1. Traffic Monitoring: Motion detection could be very handy application to control and monitor the traffic.
2. Security System: For maintaining the security of the society, firms and unwanted human activity could be detected.
3. Automatic door opening system: So if the camera will detect someone out of the door and if we allow that person so door can automatically get opened.

### **Import the Libraries**
We will start buy importing all the necessary libraries.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Why background subtraction technique?

**`Background subtraction`** is one of the most effective and easiest method to detect and extract the objects from images or video.
But why other masking methods are not that effective? Let's answer this question!

So let's assume you have one picture of highway with the cars i.e. the **foreground image** and on the other hand you have the picture of the highway without the cars i.e. this image can be used as the **background image**. Hence from this approach we can easily use segmentation technique to segment out the background image from the foreground image (differentiate cars) and then use the mask to extract the cars.


So if we can use the above method then `why is Background subtraction is needed?`
This technique comes into play when you don't have the clear background image like practically speaking it's ard to take a clear picture of the highway with the moving cars right! So in that cases background subtraction technqiues can also detect the real time moment and not only in the images. The most important feature of this algorithm is that it is faster and has the better adaptability and it is way more effiecient than the above mentioned traditional technique.

**Note:** Though OpenCV has ample of methods to solve the same problem here we will be discussing about the **`Background Subtrator MOG2`**

**First we will look for the syntax of this function**


> **`background_subtractor = cv2.createBackgroundSubtractorMOG2(history, varThreshold, detectShadows)`**

**Now, let's discuss what kind of Parameters this algorithm holds:**

* **`history`** - The is the **int** type argument that's why we gave the length of the history. The default value is **500**.


* **`varThreshold`** - This parameter is also optional and when I say optional it means that it will take the default value when it won't find a user given value. As name suggest it holds the **threshold value** which is the **squared distance** between the pixel and the algorithm to make a decision that the pixel is well defined by the model or not. The default value is **16**.


* **`detectShadows`** - This parameter hold the **boolean type** of value (True/False) and it decides that whether the algorithm will detect and **mark the shadow** or not in the output, it usually mark the shadow in **gray** color. The default value is **True** but there is also a drawback of this feature as when it is mapped to True it **hampers the speed** so for that reason if you don't want this functionality then you can set it to **False**.  


**Returns:**

* **`object`** - It will return the output in the format of MOG2 Background Subtractor

In [5]:
cap = cv2.VideoCapture('media/videos/vtest.avi')

backgroundobject = cv2.createBackgroundSubtractorMOG2( history = 2, detectShadows = True )

while(1):
    ret, frame = cap.read()  
    if not ret:
        break

    fgmask = backgroundobject.apply(frame)

    real_part = cv2.bitwise_and(frame,frame,mask=fgmask)

    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    
    stacked = np.hstack((fgmask_3,frame,real_part))
    cv2.imshow('All three',cv2.resize(stacked,None,fx=0.65,fy=0.65))
 
    k = cv2.waitKey(30) &  0xff
    if k == 27:
        break
   
cap.release()
cv2.destroyAllWindows()

Code breakdown

1. First we will be loading the video on which we want to test our algorithm. We can also test on live webcam just we need to change the paramater from **path of video** to **0**.


2. Now we will be creating the background subtractor object which is our model and take history value as 2 and detect shadow as True.


3. Then in the infinite while loop we will read the video/live stream depends on what we have chosen we will perform following functions:
    * First we will apply the **background object** on every frame of the video.
    * Then we will extract the **detected foreground** part of the frame, this is an optional step.
    * As we know we have one channeled mask so for that reason with the help of **hstack i.e. horizontal stack** we will stack all the three frames and then we will **show the image**.
    

4. At the last when the **esc** key is pressed and it is detected too then we will release the camera and destroy all the windows.

## Creating the Vehicle Detection Application.

Now we will be building our vehicle detection application using background subtractor for that we need to follow few steps so first we will see the code then it's explanation.

In [8]:
video = cv2.VideoCapture('media/videos/carsvid.wmv')

kernel = None

backgroundObject = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

while True:

    ret, frame = video.read()

    if not ret:
        break

    foreground_mask = backgroundObject.apply(frame)


    _, foreground_mask = cv2.threshold(foreground_mask, 250, 255, cv2.THRESH_BINARY)
    

    foreground_mask = cv2.erode(foreground_mask, kernel, iterations = 1)
    foreground_mask = cv2.dilate(foreground_mask, kernel, iterations = 2)

    contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frameCopy = frame.copy()
    
    # loop over each contour found in the frame.
    for cnt in contours:

        # We need to be sure about the area of the contours i.e. it should be higher than 400 to reduce the noise.
        if cv2.contourArea(cnt) > 400:
            
            # Accessing the x, y and height, width of the cars
            x, y, width, height = cv2.boundingRect(cnt)
            
            # Here we will be drawing the bounding box on the cars
            cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)
            
            # Then with the help of putText method we will write the 'Car detected' on every car with a bounding box
            cv2.putText(frameCopy, 'Car Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)

            
    foregroundPart = cv2.bitwise_and(frame, frame, mask=foreground_mask)
         
    stacked_frame = np.hstack((frame, foregroundPart, frameCopy))

    cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked_frame, None, fx=0.5, fy=0.5))

    
    k = cv2.waitKey(1) & 0xff
    
    if k == ord('q'):
        break

video.release()

cv2.destroyAllWindows()

Code breakdown:

1. Our first step will be to **load the video** which will have the cars in the moving state so that our algorithm could easily detect them.


2. Then we will set the kernel to **`None`**. We can give our own custom size but to remove the complexity of the model we will be giving it None value.


3. Now we will be initializing our **`background subtractor`** model and storing the instance of it in a variable.


4. Then after some initialization steps we will start the main work in the **infinte loop**. The first step would be to read each frame of the video and also check it under a condition that whether the frames are detected or not.


5. Then with the help of **`backgroundObject`** function we will apply it to every frame to get the **segmented mask**.


6. So from the above step we might not get the completely accurate result so for that we will be using the **thresholding** (**`cv2.thereshold`**) technique so that we could get rid of unwanted shadows.


7. Then we will perform some **`morphological operations`** so that we could have the better mask (more efficient and accurate) for that we will be using two cv2's function:
    * **`cv2.erode()`**: to improve the erosion of the frame.
    * **`cv2.dilate()`**: to improve the dilation of the frame.
    
8. Now we will detect the **contours** of the particular frame. Note: Not all the frames, for that we will first copy the frames so that we could draw the bounding box on the detected cars for that we need to **loop over each frames**.


9. Now as we have cars detected so it's time to retrieve the foreground from each frame and we will do that with the help of **`cv2.bitwise`** function.


10. Now as we are almost done so it's time to display our result but before that we will stack all the three frames i.e. **Original frame, Foreground frame** and **Annoted frame** and it will be done using **`np.hstack`** function to stack all the three frames togethere and at the last we will display the stacked format with the help of **`cv2.imshow`** function.


11. After all the processing is done we can exit from the application by pressing **q**, after that we will release the video capture object and destroy all the windows.

## Conclusion

When we will look into the documentation of OpenCV then we could find ample of algorithms for the object detection or the motion detection you can always try the different algorithm and look what fits best according to your requirements. Here I used the **`background subtraction`** because it gave me better results as well as it seems to be more reliable and easy to use.